In [3]:
# Import python packages
import requests
import sqlite3
import json
import config   # Local file config.py with BLS API key
#from api_readers import bls_api # BLS API reader
from datetime import datetime
today = datetime.now().date()

In [4]:
s_list = ['CES0000000001', 'CES3000000001', 'CES0500000003']
#s_list = ['LNS12300060', 'LES1252881600', 'LNS14000000']
dur = 65
source = 'BLS'
bls_key = config.bls_key

# Connect to SQL database and check for series
conn = sqlite3.connect('macro_data.db')
c = conn.cursor()
c.execute("SELECT name FROM sqlite_master WHERE type='table';")
res = c.fetchall()
for s_id in s_list:
    if s_id in [i[0] for i in res]:
        print('{}: Series Available!'.format(s_id))
        #ld = c.execute("select * from {}".format(s_id)).fetchall()[0][0]
        #ld = datetime.strptime(ld, '%Y-%m-%d').date()
        #if (today - ld).days > dur:
            #print('Needs updating, though...')
            #bls_api(s_id, bls_key, exists=True)
        #else:
            #print('And up-to-date!')
        #bls_api(s_id, bls_key)

        # Save and close
        #conn.commit()
    else: 
        print('{}: New table'.format(s_id))
        #bls_api(s_id, bls_key)

        # Save and close
        conn.commit()
conn.close()

CES0000000001: Series Available!
CES3000000001: Series Available!
CES0500000003: Series Available!


In [9]:
conn.close()

In [10]:
conn = sqlite3.connect('macro_data.db')
c = conn.cursor()
c.execute("SELECT name FROM sqlite_master WHERE type='table';")
c.fetchall()

[(u'epop',),
 (u'data',),
 (u'LNS12300060',),
 (u'LES1252881600',),
 (u'LNS14000000',),
 (u'CES0000000001',),
 (u'CES3000000001',),
 (u'CES0500000003',),
 (u'LNS12300061',),
 (u'LNS12300062',)]

In [4]:
conn = sqlite3.connect('macro_data.db')
c = conn.cursor()

In [13]:
c.execute("select * from {}".format('CES0500000003')).fetchall()

[(u'2017-11-01', 26.55),
 (u'2017-10-01', 26.5),
 (u'2017-09-01', 26.53),
 (u'2017-08-01', 26.42),
 (u'2017-07-01', 26.39),
 (u'2017-06-01', 26.27),
 (u'2017-05-01', 26.22),
 (u'2017-04-01', 26.18),
 (u'2017-03-01', 26.13),
 (u'2017-02-01', 26.1),
 (u'2017-01-01', 26.02),
 (u'2016-12-01', 25.98),
 (u'2016-11-01', 25.91),
 (u'2016-10-01', 25.9),
 (u'2016-09-01', 25.81),
 (u'2016-08-01', 25.74),
 (u'2016-07-01', 25.71),
 (u'2016-06-01', 25.62),
 (u'2016-05-01', 25.59),
 (u'2016-04-01', 25.54),
 (u'2016-03-01', 25.46),
 (u'2016-02-01', 25.38),
 (u'2016-01-01', 25.37),
 (u'2015-12-01', 25.26),
 (u'2015-11-01', 25.24),
 (u'2015-10-01', 25.21),
 (u'2015-09-01', 25.12),
 (u'2015-08-01', 25.1),
 (u'2015-07-01', 25.02),
 (u'2015-06-01', 24.96),
 (u'2015-05-01', 24.96),
 (u'2015-04-01', 24.89),
 (u'2015-03-01', 24.85),
 (u'2015-02-01', 24.79),
 (u'2015-01-01', 24.75),
 (u'2014-12-01', 24.64),
 (u'2014-11-01', 24.65),
 (u'2014-10-01', 24.58),
 (u'2014-09-01', 24.55),
 (u'2014-08-01', 24.55),
 (u'

In [ ]:
conn = sqlite3.connect('macro_data.db')
c = conn.cursor()
#c.execute("SELECT name FROM sqlite_master WHERE type='table';")

In [ ]:
[i[0] for i in c.fetchall()]
for s_id in s_list:
    if s_id in [i[0] for i in c.fetchall()]:
        print('{}: Series Available!'.format(s_id))

In [ ]:
c = conn.cursor()
ld = c.execute("select * from {}".format(s_id))
ld.fetchall()

In [ ]:
ld.fetchall()

In [ ]:
def bls_api(s_id, exists=False):
    """Collect data from the BLS API"""
    # BLS API v2 - access using key from registration
    api = 'https://api.bls.gov/publicAPI/v2/timeseries/data/'
    url = '{}?registrationkey={}'.format(api, config.bls_key)

    # Info for sending with post request
    headers = {'Content-type': 'application/json'}
    data = json.dumps({"seriesid": [s_id],  # BLS series ID
             "startyear": "2010", "endyear": "2017"})

    # Request json data from BLS API
    p = requests.post(url, data=data, headers=headers).json()
    print(p['status'])  # Print whether API request was successful

    # Location within API results of time series and footnotes
    r = p['Results']['series'][0]['data']

    # json date and value to list of tuples
    dates = ['{}-{}-01'.format(i['year'], i['period'][1:]) for i in r]
    data = zip(dates, [float(i['value']) for i in r])

    # Create and fill the table
    conn.execute("create table if not exists {}(Date, Value)".format(s_id))
    conn.executemany("insert into {}(Date, Value) values (?, ?)".format(s_id), data)

In [ ]:
c = conn.cursor()
c.execute("SELECT name FROM sqlite_master WHERE type='table';")

In [ ]:
c.fetchall()

In [ ]:
conn.close()

In [ ]:
print(p['status'])

In [ ]:
c = conn.cursor()
c.execute("SELECT name FROM sqlite_master WHERE type='table';")
c.fetchall()

In [ ]:
# BLS API v2 - access using key from registration
api = 'https://api.bls.gov/publicAPI/v2/timeseries/data/'
url = '{}?registrationkey={}'.format(api, config.bls_key)

# Info for sending with post request
headers = {'Content-type': 'application/json'}
data = json.dumps({"seriesid": [s_id],  # BLS series ID
         "startyear": "2010", "endyear": "2017"})

# Request json data from BLS API
p = requests.post(url, data=data, headers=headers).json()
print(p['status'])  # Print whether API request was successful

# Location within API results of time series and footnotes
r = p['Results']['series'][0]['data']

# json date and value to list of tuples
dates = ['{}-{}-01'.format(i['year'], i['period'][1:]) for i in r]
data = zip(dates, [float(i['value']) for i in r])

# Create and fill the table
conn.execute("create table if not exists {}(Date, Value)".format(s_id))
conn.executemany("insert into {}(Date, Value) values (?, ?)".format(s_id), data)

# Save and close
conn.commit()
conn.close()

In [ ]:
# Connect to SQL database
#conn = sqlite3.connect('macro_data.db')

# Create the table
conn.execute("create table if not exists {}(Date, Value)".format(s_id))

# Fill the table
conn.executemany("insert into {}(Date, Value) values (?, ?)".format(s_id), data)

# Save and close
conn.commit()
conn.close()

In [ ]:
#s_id = 'LNS12300060' # BLS series ID of interest
#s_id2 = 'LES1252881600'
#s_id3 = 'LNS14000000'

In [ ]:
#c = conn.cursor()
#print(c.execute("select * from {}".format(s_id)).fetchall())
#names = [tup[0] for tup in c.fetchall()]
#print(names)

In [ ]:
#c.fetchall()

In [ ]:
#con = sqlite3.connect('macro_data.db')
#c = conn.cursor()
#c.execute("SELECT name FROM sqlite_master WHERE type='table';")
#print(cursor.fetchall())

In [ ]:
#print(c.fetchall())
#table = c.execute("select * from {}".format(s_id)).fetchall()

#if len(table) == 0:
    #Get full data and create new table
    
#elif 

#print(table)
#print(len(table))
# Save string with data of latest datapoint
#ldate = table[0][0]